
# 1. Deposit Model and Lineage in SageMaker Model Registry
# 2. Deploy to an endpoint

Once a useful model has been trained and its artifacts properly associated, the next step is to save the model in a registry for future reference and possible deployment.

References: 
- https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry.html
- https://sagemaker-examples.readthedocs.io/en/latest/end_to_end/fraud_detection/2-lineage-train-assess-bias-tune-registry-e2e.html

#### What Is Model Registry in SageMaker?
SageMaker Model Registry is a centralized repository to:

- Catalog models for production.
- Manage model versions.
- Associate metadata, such as training metrics, with a model.
- View information from Amazon SageMaker Model Cards in your registered models.
- View model lineage for traceability and reproducibility.
- Define a staging construct that models can progress through for your model lifecycle.
- Manage the approval status of a model.
- Deploy models to production.
- Automate model deployment with CI/CD.

Share models with other users.
#### But When Should You Register a Model?

📌 1. Versioning

- Model Registry keeps track of multiple versions.
- You can roll back to a previous version easily.
- Each version stores metadata like creation time, metrics, and approval status.

📌 2. Approval Workflow
- You can set a model’s status as: PendingManualApproval (Default), Approved, Rejected.
- Good for human review, audit, or CI/CD pipelines.
- Prevents accidental deployment of unverified models.

📌 3. Audit & Traceability
- Logs who registered the model, which training job it came from, and what data was used.
- Helps in regulated environments like finance, healthcare, or defense.

📌 4. Production-readiness
- Enables stage promotion (e.g., dev → staging → production).
- Integrates with SageMaker Pipelines for full MLOps automation.

📌 5. Team collaboration
- Shared model group across teams lets everyone see:
- Latest approved model
- Metrics, parameters, metadata
- History of changes

References:
 - https://sagemaker-examples.readthedocs.io/en/latest/end_to_end/fraud_detection/2-lineage-train-assess-bias-tune-registry-e2e.html

In [1]:
import sagemaker
import boto3

/opt/conda/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
REGION = sagemaker.Session().boto_region_name
boto_session = boto3.Session(region_name=REGION)
s3_client = boto_session.client('s3')
sagemaker_client = boto3.client('sagemaker', region_name=REGION)

sagemaker_session = sagemaker.session.Session(boto_session=boto_session, sagemaker_client=sagemaker_client)

BUCKET = sagemaker_session.default_bucket()
ROLE = sagemaker.get_execution_role()
PREFIX = "FraudDetection_AutoInsurance"
print(REGION)
print(BUCKET)
print(ROLE)

us-east-1
sagemaker-us-east-1-205930620783
arn:aws:iam::205930620783:role/service-role/AmazonSageMaker-ExecutionRole-20250401T145997


### If no model is found with the name, then create it from the training job


In [5]:
model_name = "sagemaker-xgboost-v3"
training_job_name = 'sagemaker-xgboost-2025-06-11-13-18-27-251'

training_job_info = sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
print(training_job_info['TrainingJobName'])
print(training_job_info["AlgorithmSpecification"]["TrainingImage"])
training_job_info["ModelArtifacts"]["S3ModelArtifacts"]    

models = sagemaker_client.list_models(NameContains=model_name)['Models']
print(models)
if not models:
    # If no model is found with the name, then create it from the training job
    
    model1 = sagemaker_session.create_model_from_job(
        name=model_name, 
        training_job_name=training_job_name, 
        role=ROLE, 
        image_uri=training_job_info["AlgorithmSpecification"]["TrainingImage"]
    )
    print("Creating Model")

sagemaker-xgboost-2025-06-11-13-18-27-251
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3
[{'ModelName': 'sagemaker-xgboost-v3', 'ModelArn': 'arn:aws:sagemaker:us-east-1:205930620783:model/sagemaker-xgboost-v3', 'CreationTime': datetime.datetime(2025, 6, 11, 14, 2, 6, 361000, tzinfo=tzlocal())}]


## Now Registered the model in the Model Registry

### Create Model Package Group
A Model Package Groups holds multiple versions or iterations of a model. Though it is not required to create them for every model in the registry, they help organize various models which all have the same purpose and provide automatic versioning.

The following code will create a model package group i.e. "SageMaker Studio > Models > Registered Models > FraudDetection AutoInsurance"


In [6]:
model_package_group_name = PREFIX.replace('_','-')
#sagemaker_client.list_model_package_groups(NameContains=model_package_group_name)
matching_group = sagemaker_client.list_model_package_groups(NameContains=model_package_group_name)['ModelPackageGroupSummaryList']
if matching_group:
    print(f'Using existing Model Package Group: {model_package_group_name}')
else:
    mpg_input_dict = {'ModelPackageGroupName': model_package_group_name, 'ModelPackageGroupDescription': 'Auto Insurance E2E'}
    mpg_response = sagemaker_client.create_model_package_group(**mpg_input_dict)
    print(f'Create Model Package Group {model_package_group_name}: SUCCESSFUL')

Using existing Model Package Group: FraudDetection-AutoInsurance


### Create Model Package

#### Define the model inference specification
- Containers: i.e. container image, s3 location of model output i.e. model.tar.gz
- Request Type: What format the endpoint will accept the input for inference
- Response Type: Response format.

**Point to note:** This settings serve a declarative/documentation role, not an enforcement one at runtime for SageMaker’s Predictor client. So you need to define serializers and deserializers at the time of predictions

In [7]:
model_package_inference_specification = {
    "InferenceSpecification": {
        "Containers": [{
        "Image": training_job_info["AlgorithmSpecification"]["TrainingImage"],
        "ModelDataUrl": training_job_info["ModelArtifacts"]["S3ModelArtifacts"]
        }],
        "SupportedContentTypes": ["text/csv"],
        "SupportedResponseMIMETypes": ["text/csv"],
    }
}
print(model_package_inference_specification)

{'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3', 'ModelDataUrl': 's3://sagemaker-us-east-1-205930620783/FraudDetection_AutoInsurance/training_jobs/sagemaker-xgboost-2025-06-11-13-18-27-251/output/model.tar.gz'}], 'SupportedContentTypes': ['text/csv'], 'SupportedResponseMIMETypes': ['text/csv']}}


In [8]:

create_model_package_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageDescription": "Model to detect fraud in auto-insurance",
    "ModelApprovalStatus": "PendingManualApproval" # Defaul value
}
create_model_package_input_dict.update(model_package_inference_specification)

create_model_package_response = sagemaker_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_model_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

ModelPackage Version ARN : arn:aws:sagemaker:us-east-1:205930620783:model-package/FraudDetection-AutoInsurance/2


#### List and Describe Model Packages

In [9]:
print(sagemaker_client.list_model_package_groups())
print()
print(sagemaker_client.list_model_packages(ModelPackageGroupName='FraudDetection-AutoInsurance'))
print()
print(sagemaker_client.describe_model_package(ModelPackageName='arn:aws:sagemaker:us-east-1:205930620783:model-package/FraudDetection-AutoInsurance/1'))
print()
#print(sagemaker_client.describe_model_package(ModelPackageName='arn:aws:sagemaker:us-east-1:205930620783:model-package/FraudDetection-AutoInsurance/1')['ModelPackageName'])



{'ModelPackageGroupSummaryList': [{'ModelPackageGroupName': 'FraudDetection-AutoInsurance', 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:205930620783:model-package-group/FraudDetection-AutoInsurance', 'ModelPackageGroupDescription': 'Auto Insurance E2E', 'CreationTime': datetime.datetime(2025, 6, 12, 7, 8, 0, 174000, tzinfo=tzlocal()), 'ModelPackageGroupStatus': 'Completed'}], 'ResponseMetadata': {'RequestId': '539ddc35-b5c0-4c69-9d00-6e28f23c452f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '539ddc35-b5c0-4c69-9d00-6e28f23c452f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '328', 'date': 'Sat, 14 Jun 2025 07:06:29 GMT'}, 'RetryAttempts': 0}}

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'FraudDetection-AutoInsurance', 'ModelPackageVersion': 2, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:205930620783:model-package/FraudDetection-AutoInsurance/2', 'ModelPackageDescription': 'Model to detect fraud in auto-insurance', 'CreationTime

### Model Registry Collections
You can use Collections to group registered models that are related to each other and organize them in hierarchies to improve model discoverability at scale. With Collections, you can organize registered models that are associated with one another. For example, you could categorize your models based on the domain of the problem they solve as Collections titled NLP-models, CV-models, or Speech-recognition-models. To organize your registered models in a tree structure, you can nest Collections within each other.

**For now we are skipping it**

## Model Deployment to an endpoint for real-time inference

In [10]:
## First Approve the model to deploy
update_model_package_dict = {
    'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:205930620783:model-package/FraudDetection-AutoInsurance/1',
    'ModelApprovalStatus': 'Approved'
}
sagemaker_client.update_model_package(**update_model_package_dict)

{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:205930620783:model-package/FraudDetection-AutoInsurance/1',
 'ResponseMetadata': {'RequestId': 'fe1d04a2-0563-4040-bb78-131d25a8010f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fe1d04a2-0563-4040-bb78-131d25a8010f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '107',
   'date': 'Sat, 14 Jun 2025 07:06:43 GMT'},
  'RetryAttempts': 0}}

In [17]:
endpoint_name = f"{model_package_group_name}-endpoint"
print(endpoint_name)
endpoint_instance_count = 1
endpoint_instance_type = "ml.t2.medium"#"ml.m4.xlarge"


FraudDetection-AutoInsurance-endpoint


In [18]:
from sagemaker import ModelPackage

model_package = ModelPackage(
    role=ROLE, 
    model_package_arn='arn:aws:sagemaker:us-east-1:205930620783:model-package/FraudDetection-AutoInsurance/1',
    sagemaker_session=sagemaker_session
)

predictor = model_package.deploy(
    endpoint_name=endpoint_name,
    instance_type=endpoint_instance_type,
    initial_instance_count=1
)

[06/14/25 07:12:20] INFO     Creating model with name:                                              ]8;id=395117;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=316181;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4094\4094]8;;\
                             FraudDetection-AutoInsurance-2025-06-14-07-12-20-391                                  

[06/14/25 07:12:21] INFO     Creating endpoint-config with name                                     ]8;id=201311;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=657529;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#5937\5937]8;;\
                             FraudDetection-AutoInsurance-endpoint                                                 

                    INFO     Creating endpoint with name FraudDetection-AutoInsurance-endpoint      ]8;id=382555;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=251541;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4759\4759]8;;\

-----------!

In [20]:
help(model_package.deploy)

Help on method deploy in module sagemaker.model:

deploy(initial_instance_count=None, instance_type=None, serializer=None, deserializer=None, accelerator_type=None, endpoint_name=None, tags=None, kms_key=None, wait=True, data_capture_config=None, async_inference_config=None, serverless_inference_config=None, volume_size=None, model_data_download_timeout=None, container_startup_health_check_timeout=None, inference_recommendation_id=None, explainer_config=None, accept_eula: Optional[bool] = None, endpoint_logging=False, resources: Optional[sagemaker.compute_resource_requirements.resource_requirements.ResourceRequirements] = None, endpoint_type: sagemaker.enums.EndpointType = <EndpointType.MODEL_BASED: 'ModelBased'>, managed_instance_scaling: Optional[str] = None, inference_component_name=None, routing_config: Optional[Dict[str, Any]] = None, model_reference_arn: Optional[str] = None, inference_ami_version: Optional[str] = None, **kwargs) method of sagemaker.model.ModelPackage instance
  